# **CP-IOT: Análise Integrada de Vinho (Visão Computacional e Dados Tabulares)**

## Configuração Inicial e Download do Dataset

In [ ]:
import os
from ultralytics import YOLO
from roboflow import Roboflow

# ===================================================================
# PASSO 1: PREPARAÇÃO DE DADOS E DOWNLOAD (Ferramenta: Roboflow)
# Objetivo: Trazer o dataset de rótulos de vinho já rotulado e formatado
# ===================================================================

# Instalamos as bibliotecas necessárias para rodar o YOLOv8 e fazer o download do Roboflow
print("Instalando ultralytics e roboflow...")
!pip install ultralytics roboflow

# Estamos usando variaveis para evitar o versionamento da API Key
try:
    from google.colab import userdata
    ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
except ImportError:
    ROBOFLOW_API_KEY = os.environ.get("ROBOFLOW_API_KEY")

# Usamos a API Key para nos autenticar e acessar o dataset público do Roboflow Universe
# A API Key garante que o download venha no formato correto (YOLOv8)
rf = Roboflow(api_key=ROBOFLOW_API_KEY)


# Definimos o projeto, a versão e iniciamos o download do dataset no formato YOLOv8
project = rf.workspace("wine-label").project("wine-label-detection")
version = project.version(7)
dataset = version.download("yolov8")

print("Download do Dataset Roboflow concluído. Pasta criada: Wine-Label-Detection-7")

## Treinamento do Modelo (YOLOv8)

In [5]:
# ===================================================================
# PASSO 2: CONFIGURAÇÃO E INÍCIO DO TREINAMENTO (Ferramenta: YOLOv8)
# Objetivo: Fazer o Fine-Tuning do modelo SOTA (State-of-the-Art).
# ===================================================================

# Definimos o caminho para o arquivo de configuração YAML que o Roboflow gerou.
DATASET_DIR = 'Wine-Label-Detection-7'
CONFIG_PATH = os.path.join(DATASET_DIR, 'data.yaml')

# Carregamos o modelo base 'yolov8n.pt'
model = YOLO('yolov8n.pt')

# Iniciamos o treinamento (Fine-Tuning)
print(f"\nIniciando treinamento do YOLOv8 em 50 épocas...")
results = model.train(
    data=CONFIG_PATH,         # informa o caminho do dataset (YAML)
    epochs=50,                # número de ciclos de aprendizado
    imgsz=640,                # resolução de entrada
    batch=16,                 # quantidade de imagens processadas por vez
    name='yolov8_wine_labels_final' # nome para salvar os resultados (runs/detect/...)
)

print("\nTreinamento concluído. O melhor modelo foi salvo em: /content/runs/detect/yolov8_wine_labels_final/weights/best.pt")

Instalando dependências...
loading Roboflow workspace...
loading Roboflow project...

Iniciando treinamento com o arquivo YAML: Wine-Label-Detection-7/data.yaml
Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Wine-Label-Detection-7/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.

In [6]:
PASTA_IMAGENS_TESTE = 'Wine-Label-Detection-7/test/images/'

# listando os nomes dos arquivos
!ls $PASTA_IMAGENS_TESTE | head -n 5

100140_jpg.rf.8111877cb22c418e6b456d8c8c2121bd.jpg
101108_jpg.rf.eccf21abf10389bcd245e574511a249f.jpg
102077_jpg.rf.4b10189136ddc5746984fefe180d9d02.jpg
102737_jpg.rf.bc5b3e487f718d0b21d3bdecc42624e2.jpg
102896_jpg.rf.06ef074880a1ffeb4a75e3e9f2a584c3.jpg


## Avaliação e Demonstração

In [26]:
import os
from ultralytics import YOLO

# PASSO 1: CARREGAR O MELHOR MODELO TREINADO

# Carregamos o arquivo de pesos 'best.pt' salvo apos as 50 épocas de treinamento
# É o nosso modelo final para detecção de rótulos de vinho
best_model_path = '/content/runs/detect/yolov8_wine_labels_final/weights/best.pt'
trained_model = YOLO(best_model_path)

# PASSO 2: CONFIGURAR O ARQUIVO DE TESTE

# Definimos o diretório raiz do dataset que o Roboflow baixou
DATASET_DIR = 'Wine-Label-Detection-7'

# Nome da imagem que vamos usar para a demonstração (escolhida na pasta 'test/images')
NOME_DO_ARQUIVO_TESTE = '102077_jpg.rf.4b10189136ddc5746984fefe180d9d02.jpg'

# Montamos o caminho completo para a imagem de teste
PASTA_IMAGENS_TESTE = os.path.join(DATASET_DIR, 'test/images/')
TEST_IMAGE_PATH = os.path.join(PASTA_IMAGENS_TESTE, NOME_DO_ARQUIVO_TESTE)


# PASSO 3: RODAR A PREDIÇÃO (DETECÇÃO)

print(f"Executando a detecção na imagem: {TEST_IMAGE_PATH}")

trained_model.predict(
    source=TEST_IMAGE_PATH,
    conf=0.25,  # Definimos a confiança minima (25%) para exibir as caixas delimitadoras
    save=True,  # Isso garante que a imagem com as caixas de detecção sera salva
    name='wine_prediction_demo' # Nome para a pasta de resultados da demonstração
)

# Indicamos onde a imagem final (o artefato visual do projeto) foi salva no Colab
print("\nResultado da detecção (imagem com caixas delimitadoras) salvo em: /content/runs/detect/wine_prediction_demo/")

Executando a detecção na imagem: Wine-Label-Detection-7/test/images/102077_jpg.rf.4b10189136ddc5746984fefe180d9d02.jpg

image 1/1 /content/Wine-Label-Detection-7/test/images/102077_jpg.rf.4b10189136ddc5746984fefe180d9d02.jpg: 640x640 3 CountryCountrys, 1 Distinct Logo, 1 Maker-Name, 2 TypeWine Types, 9.8ms
Speed: 3.8ms preprocess, 9.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/detect/wine_prediction_demo19

Resultado da detecção (imagem com caixas delimitadoras) salvo em: /content/runs/detect/wine_prediction_demo/
